In [ ]:
%load_ext kedro.ipython

In [ ]:
import re

import pandas as pd

In [ ]:
# ruff: noqa: F821
catalog.list()

In [ ]:
# ruff: noqa: F821
merged_data = catalog.load("merged_data")
merged_data = merged_data.drop(["to_remove", "remove_type"], axis=1)
merged_data

In [ ]:
# ruff: noqa: F821
whitelist = catalog.load("params:whitelist")
whitelist

In [ ]:
merged_data["to_remove"] = False
merged_data["remove_type"] = None

In [ ]:
merged_data["content_category"].value_counts().sort_values(ascending=False)

## NaN


In [ ]:
merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: x.isna().sum()
).sort_values(ascending=False)

In [ ]:
# Get indexes for all NaN content body
na_indexes = merged_data[merged_data["content_body"].isna()].index

merged_data.loc[na_indexes, "to_remove"] = True
merged_data.loc[na_indexes, "remove_type"] = "NaN"

## Excel Error


In [ ]:
merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: (x.str.contains("Value exceeded maximum cell size")).sum()
).sort_values(ascending=False)

In [ ]:
content_category = "live-healthy-articles"

# Define the regex pattern
pattern = re.compile(r"(Value exceeded maximum cell)")


# Function to apply regex and return True or False
def apply_regex_v1(x):
    if pd.isna(x):
        return False
    return bool(pattern.search(str(x)))


filtered = merged_data[merged_data["content_body"].apply(lambda x: apply_regex_v1(x))]
filtered.query(f"content_category == '{content_category}'")

In [ ]:
filtered.groupby(by="content_category").size().sort_values(ascending=False)

In [ ]:
# Either NaN or Value exceeded maximum cell
merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: (x.isna() | x.str.contains("Value exceeded maximum cell")).sum()
).sort_values(ascending=False)

In [ ]:
# All Excel Error indexes
excel_error = "Value exceeded maximum cell size"
excel_error_indexes = merged_data.query(
    f"content_body.str.contains('{excel_error}', na=False)"
).index

merged_data.loc[excel_error_indexes, "to_remove"] = True
merged_data.loc[excel_error_indexes, "remove_type"] = "Excel Error"

In [ ]:
excel_error_indexes

## No Tags


In [ ]:
# Define the regex pattern
pattern = re.compile(r"(<[div|p|h2].*?>)")


# Function to apply regex and return True or False
def apply_regex_v2(x):
    if pd.isna(x) or "Value exceeded maximum cell size" in str(x):
        return True
    return bool(pattern.search(str(x)))


# Group by content_category and apply the regex
grouped = merged_data.groupby(by="content_category")["content_body"].apply(
    lambda x: ~x.apply(apply_regex_v2)
)

grouped.groupby(by="content_category").value_counts().loc[:, True].sort_values(
    ascending=False
)

In [ ]:
# All content without HTML tags indexes
no_tags_indexes = merged_data[
    ~merged_data.query("content_category.notna()")["content_body"].apply(
        lambda x: apply_regex_v2(x)
    )
].index

merged_data.loc[no_tags_indexes, "to_remove"] = True
merged_data.loc[no_tags_indexes, "remove_type"] = "No HTML Tags"

## No Extracted Content


In [ ]:
# All content ids without extracted content
no_extracted_content_ids = set(
    merged_data[merged_data["extracted_content_body"] == ""].id.to_list()
).difference(set(whitelist))

# All content without extracted content indexes
no_extracted_content_indexes = merged_data.query(
    "id in @no_extracted_content_ids"
).index

merged_data.loc[no_extracted_content_indexes, "to_remove"] = True
merged_data.loc[no_extracted_content_indexes, "remove_type"] = "No Extracted Content"

In [ ]:
merged_data.groupby(by="content_category")["remove_type"].apply(
    lambda x: (x == "No Extracted Content").sum()
).sort_values(ascending=False)

## Duplicated Content

**Note:** Done across content categories <u>not</u> together, merged.


In [ ]:
duplicated = pd.DataFrame()

for content_category in merged_data["content_category"].unique():
    tmp = merged_data.query("content_category == @content_category")
    tmp = tmp[
        (tmp["extracted_content_body"].duplicated())
        & (tmp["extracted_content_body"].notna())
        & (tmp["extracted_content_body"] != "")
        & (~tmp["to_remove"])
    ]

    duplicated = pd.concat([duplicated, tmp], axis=0)

duplicated

In [ ]:
merged_data["duplicates_with"] = None

for i in range(len(duplicated)):
    tmp = merged_data[
        (
            merged_data["extracted_content_body"]
            == duplicated.iloc[i]["extracted_content_body"]
        )
        & (merged_data["content_category"] == duplicated.iloc[i]["content_category"])
    ]

    indexes = tmp.index.tolist()

    for index in indexes:
        curr_id = merged_data.iloc[index]["id"]

        # Ignore whitelisted articles
        if curr_id in whitelist:
            continue

        all_ids = merged_data.loc[indexes, "id"].to_list()

        # Remove current ID
        all_ids.remove(curr_id)

        if not merged_data.iloc[index]["to_remove"]:
            # Update `to_remove`
            merged_data.at[index, "to_remove"] = True

            # Update `remove_type` to "Duplicated Content"
            merged_data.at[index, "remove_type"] = "Duplicated Content"

            # Update column `duplicates_with` to article IDs
            merged_data.at[index, "duplicates_with"] = all_ids

In [ ]:
tmp = merged_data.query("remove_type == 'Duplicated Content'")
tmp.groupby(by="content_category").size().sort_values(ascending=False)

## Duplicated URLs

**Note:** Done across content categories <u>not</u> together, merged.


In [ ]:
duplicated = pd.DataFrame()

for content_category in merged_data["content_category"].unique():
    tmp = merged_data.query("content_category == @content_category")
    tmp = tmp[
        (tmp["full_url"].duplicated()) & (tmp["full_url"].notna()) & (~tmp["to_remove"])
    ]

    duplicated = pd.concat([duplicated, tmp], axis=0)

duplicated

In [ ]:
for i in range(len(duplicated)):
    tmp = merged_data[
        (merged_data["full_url"] == duplicated.iloc[i]["full_url"])
        & (merged_data["content_category"] == duplicated.iloc[i]["content_category"])
    ]

    indexes = tmp.index.tolist()

    for index in indexes:
        curr_id = merged_data.iloc[index]["id"]

        # Ignore whitelisted articles
        if curr_id in whitelist:
            continue

        all_ids = merged_data.loc[indexes, "id"].to_list()

        # Remove current ID
        all_ids.remove(curr_id)

        if not merged_data.iloc[index]["to_remove"]:
            # Update `to_remove`
            merged_data.at[index, "to_remove"] = True

            # Update `remove_type` to "Duplicated URL"
            merged_data.at[index, "remove_type"] = "Duplicated URL"

            # Update column `duplicates_with` to article IDs
            merged_data.at[index, "duplicates_with"] = all_ids

In [ ]:
tmp = merged_data.query("remove_type == 'Duplicated URL'")
tmp.groupby(by="content_category").size().sort_values(ascending=False)

## Below Word Count


In [ ]:
# ruff: noqa: F821
word_count_cutoff = catalog.load("params:word_count_cutoff")
word_count_cutoff

In [ ]:
# Assuming merged_data is your DataFrame and `word_count_cutoff` is defined
indices = merged_data.query(
    "extracted_content_body.notna() "
    "and remove_type != 'Duplicated Content' "
    "and remove_type != 'Duplicated URL'"
)["extracted_content_body"].apply(
    lambda x: len(x.split()) > 0 and len(x.split()) <= word_count_cutoff
)

In [ ]:
# Get the indices of the True values
below_word_count_indexes = indices[indices].index

# All content ids below word count cutoff
below_word_count_ids = set(
    merged_data.iloc[below_word_count_indexes].id.to_list()
).difference(set(whitelist))

# All content below word count cutoff indexes
below_word_count_indexes = merged_data.query("id in @below_word_count_ids").index

merged_data.loc[below_word_count_indexes, "to_remove"] = True
merged_data.loc[below_word_count_indexes, "remove_type"] = "Below Word Count"

In [ ]:
tmp = merged_data.query("remove_type == 'Below Word Count'")
tmp.groupby(by="content_category").size().sort_values(ascending=False)

---


## Examples (Excel Error)


In [ ]:
content_category = "diseases-and-conditions"

tmp = merged_data.query(
    f"content_category == '{content_category}' and remove_type == 'Excel Error'"
)
tmp

In [ ]:
tmp["content_body"].values.tolist()

In [ ]:
tmp["title"].values.tolist()

In [ ]:
tmp["full_url"].values.tolist()

## Examples (No Tags)


In [ ]:
content_category = "cost-and-financing"

tmp = merged_data.query(
    f"content_category == '{content_category}' and remove_type == 'No HTML Tags'"
)
tmp

In [ ]:
print(tmp["content_body"].values.tolist()[0])

In [ ]:
tmp["title"].values.tolist()

In [ ]:
tmp["full_url"].values.tolist()

## Examples (No Extracted Content)


In [ ]:
merged_data.query(f"content_category == '{content_category}'")[
    "remove_type"
].value_counts()

In [ ]:
content_category = "support-group-and-others"

tmp = merged_data.query(
    f"content_category == '{content_category}' and remove_type == 'No Extracted Content'"
)
tmp

In [ ]:
tmp["content_body"].values.tolist()

In [ ]:
tmp["title"].values.tolist()

In [ ]:
tmp["full_url"].values.tolist()

## Examples (Duplicated Content)


In [ ]:
merged_data.query("remove_type == 'Duplicated Content'")["title"].values.tolist()

In [ ]:
merged_data.query("remove_type == 'Duplicated Content'")["full_url"].values.tolist()

## Examples (Duplicated URL)


In [ ]:
merged_data.query("remove_type == 'Duplicated URL'")["title"].values.tolist()

In [ ]:
merged_data.query("remove_type == 'Duplicated URL'")["full_url"].values.tolist()

## Examples (Below Word Count)


In [ ]:
merged_data["remove_type"].value_counts()

In [ ]:
tmp = merged_data.query(
    "extracted_content_body.notna() "
    "and remove_type != 'Duplicated Content' "
    "and remove_type != 'Duplicated URL'"
)
tmp = tmp[
    tmp["extracted_content_body"].apply(
        lambda x: len(x.split()) > 0 and len(x.split()) <= word_count_cutoff
    )
]

In [ ]:
content_category = "diseases-and-conditions"

sampled = tmp.query(f"content_category == '{content_category}'").sample(1)

print(sampled.index[0])
print(sampled["full_url"].values[0])
print(sampled["title"].values[0])
print(sampled["extracted_content_body"].values[0])

## Save Excel with `remove_type` column for VML


In [ ]:
merged_data.shape

In [ ]:
merged_data.groupby(by="content_category")["to_remove"].value_counts().loc[
    :, True
].sort_values(ascending=False)

In [ ]:
(
    merged_data.groupby(by="content_category")["to_remove"]
    .value_counts(normalize=True)
    .loc[:, True]
    .sort_values(ascending=False)
    * 100.00
).round(2)

In [ ]:
merged_data["remove_type"].value_counts()

In [ ]:
merged_data["remove_type"].value_counts().sum()

In [ ]:
merged_data.columns

In [ ]:
flag_for_removal_by_type = merged_data[
    [
        "id",
        "content_category",
        "pr_name",
        "content_name",
        "title",
        "full_url",
        "content_body",
        "extracted_content_body",
        "to_remove",
        "remove_type",
        "duplicates_with",
    ]
]

In [ ]:
# # # ruff: noqa: F821
# catalog.save("flag_for_removal_by_type", flag_for_removal_by_type)

In [ ]:
# ruff: noqa: F821
flag_for_removal_by_type = catalog.load("flag_for_removal_by_type")

In [ ]:
flag_for_removal_by_type.query("to_remove == True")["content_category"].value_counts()